In [1]:
import numpy as np
from PIL import Image
import random
import cv2
from utils import *
from tqdm import trange, tqdm
import PIL.ImageOps
import base64
from Crypto.Cipher import AES
from Crypto.Cipher import DES
from Crypto.Cipher import DES3
from Crypto import Random
import os
import sys

In [2]:
mssg_image = Image.open("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/IMG_5356.jpg")
w,h = mssg_image.size

In [3]:
def random_arr(n,cons):
    ran = np.array([])
    c = 0
    while c < cons:
        rn = random.randint(0,n-1)
        if rn not in ran:
            ran = np.append(ran,rn)
            c+=1
    return ran

def to_image(strm,r,c):
    red,grn,blu = strm[:,0:8],strm[:,8:16],strm[:,16:24]
    r_const = np.zeros(red.shape[0])
    g_const = np.zeros(grn.shape[0])
    b_const = np.zeros(blu.shape[0])
    
    for i in range(len(red)):
        r_const[i] = red[i].dot(2**np.arange(red[i].size)[::-1])
    for i in range(len(grn)):
        g_const[i] = grn[i].dot(2**np.arange(grn[i].size)[::-1])
    for i in range(len(blu)):
        b_const[i] = blu[i].dot(2**np.arange(blu[i].size)[::-1])
    
    r_reshaped,g_reshaped,b_reshaped = r_const.reshape((r,c)), g_const.reshape((r,c)), b_const.reshape((r,c))
    return Image.merge("RGB", (Image.fromarray(np.uint8(r_reshaped)), Image.fromarray(np.uint8(g_reshaped)), Image.fromarray(np.uint8(b_reshaped))))

def NSharing(img, k, n):
    r,c = img.size
    re, gr, bl = Image.Image.split(img)
    unpack_r,unpack_g,unpack_b = np.unpackbits(re,axis=1).reshape(r*c,8), np.unpackbits(gr,axis=1).reshape(r*c,8), np.unpackbits(bl,axis=1).reshape(r*c,8) 
    strm_img = np.concatenate((unpack_r, unpack_g, unpack_b), axis=1)
    bit_row_sz,bit_col_sz = strm_img.shape[0],strm_img.shape[1]
    tmp = np.zeros((n,bit_row_sz,bit_col_sz))        
    for rw in trange(bit_row_sz, desc="Making Shares"):
        for cl in range(bit_col_sz):
            if strm_img[rw,cl] ==1:
                rand = random_arr(n, n - k + 1)
                for sh in range((n - k + 1)):
                    tmp[int(rand[sh]),bit_row_sz-rw-1,bit_col_sz-cl-1] = 1                  
    for x in range(n):                    
        sh_img = to_image(tmp[x, :, :],r,c)
        sh_img.save("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Shares/share_"+str(x)+".png")

def decrypt_share(shares,r,c,n,folder):        
    temp = np.zeros((r*c, 24), dtype=np.uint8)
    try:
        for si in range(n):            
            sr = shares[si]           
            re, gr, bl = Image.Image.split(sr)
            unpack_r,unpack_g,unpack_b = np.unpackbits(re, axis=1).reshape(r * c, 8),np.unpackbits(gr, axis=1).reshape(r * c, 8), np.unpackbits(bl, axis=1).reshape(r * c, 8)
            reconst_img = np.concatenate((unpack_r, unpack_g, unpack_b), axis=1)          
            temp = (temp | np.flip(reconst_img))        
            decrypt_cover = to_image(temp,c,r)
        decrypt_cover.save("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/"+str(folder)+"/org_img_from_shares.png")
        #decrypt_cover.save("/Users/himanshuchuri/Desktop/Cryptography/Project/Test_Answers/org_img_from_shares.png")
    except:
        print("No of Shares not matching!")

# Generate Shares

In [4]:
k = 5
n = 6
cons = n - k + 1
NSharing(mssg_image, k, n)

Making Shares: 100%|████████████████| 2342673/2342673 [05:16<00:00, 7408.22it/s]


# AES 16 byte

In [5]:
k = os.urandom(16)
iv = os.urandom(16)

path_to_dir = "/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Shares"
n =0

for file in tqdm(os.scandir(path_to_dir),desc="AES encryption start"):
    if file.name != '.DS_Store':
        with open(file.path, "rb") as img_file:
            i = base64.b64encode(img_file.read())
        cipher = AES.new(k, AES.MODE_CFB, iv)
        ciphered_img = cipher.encrypt(i)
        with open("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_AES_enc/AES_enc_"+str(n),'wb') as f2:
            f2.write(ciphered_img)
            n+=1

AES encryption start: 7it [00:01,  4.16it/s]


In [6]:
path_to_dir = "/Users/himanshuchuri/Desktop/Cryptography/Project/Save_AES_enc"

#path_to_dir = "/Users/himanshuchuri/Desktop/Cryptography/Project/Save_AES_enc"
n =0

for file in tqdm(os.scandir(path_to_dir),desc="AES decryption start"):
    if file.name != '.DS_Store':
        with open(file.path, "rb") as img_file:
            i = img_file.read()
        cipher = AES.new(k, AES.MODE_CFB, iv)
        dec_img = cipher.decrypt(i)
        decode_img = base64.decodebytes(dec_img)
        with open("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_AES_dec/AES_dec_"+str(n)+'.png','wb') as f2:
            f2.write(decode_img)
            n+=1

AES decryption start: 7it [00:01,  4.19it/s]


# DES

In [7]:
k = os.urandom(8)
iv = os.urandom(8)

path_to_dir = "/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Shares"
n =0

for file in tqdm(os.scandir(path_to_dir),desc="DES encryption start"):
    if file.name != '.DS_Store':
        with open(file.path, "rb") as img_file:
            i = img_file.read()
        cipher = DES.new(k, DES.MODE_CFB, iv)
        enc_img = cipher.encrypt(i)
        with open("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_DES_enc/DES_enc_"+str(n),'wb') as f2:
            f2.write(enc_img)
            n+=1

DES encryption start: 7it [00:02,  2.74it/s]


In [8]:
path_to_dir = "/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_DES_enc"

n =0

for file in tqdm(os.scandir(path_to_dir),desc="DES decryption start"):
    if file.name != '.DS_Store':
        with open(file.path, "rb") as img_file:
            i = img_file.read()
        cipher = DES.new(k, DES.MODE_CFB, iv)
        dec_img = cipher.decrypt(i)
        with open("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_DES_dec/DES_dec_"+str(n)+'.png','wb') as f2:
            f2.write(dec_img)
            n+=1

DES decryption start: 7it [00:02,  3.22it/s]


# Triple DES

In [9]:
k = os.urandom(24)
iv = os.urandom(8)

path_to_dir = "/Users/himanshuchuri/Desktop/Cryptography/Project/Shares"
n =0

for file in tqdm(os.scandir(path_to_dir),desc="TDES encryption start"):
    if file.name != '.DS_Store':
        with open(file.path, "rb") as img_file:
            i = img_file.read()
        cipher = DES3.new(k, DES3.MODE_OFB,iv)
        enc_img = cipher.encrypt(i)
        with open("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_TDES_enc/TDES_enc_"+str(n),'wb') as f2:
            f2.write(enc_img)
            n+=1

TDES encryption start: 7it [00:00,  7.56it/s]


In [10]:
path_to_dir = "/Users/himanshuchuri/Desktop/Cryptography/Project/Save_TDES_enc"

n=0

for file in tqdm(os.scandir(path_to_dir),desc="TDES decryption start"):
    if file.name != '.DS_Store':
        with open(file.path, "rb") as img_file:
            i = img_file.read()
        cipher = DES3.new(k, DES3.MODE_OFB,iv)
        dec_img = cipher.decrypt(i)
        with open("/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_TDES_dec/TDES_dec_"+str(n)+'.png','wb') as f2:
            f2.write(dec_img)
            n+=1

TDES decryption start: 7it [00:00,  7.51it/s]


# DeSharing AES

In [11]:
dirx = "/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_AES_dec/"
#dirx = "/Users/himanshuchuri/Desktop/Cryptography/Project/Test/"

In [12]:
shr,c = [],n
for file in os.listdir(dirx):
    if file!=".DS_Store" and os.path.isfile(os.path.join(dirx, file)):
        shr.append(Image.open(os.path.join(dirx, file)))

In [13]:
rw,cl = np.asarray(shr[0]).shape[:2]
decrypt_share(shr,rw,cl,c,"Deshare_AES")

# DeSharing DES

In [14]:
dirx = "/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_DES_dec/"

In [15]:
shr,c = [],n
for file in os.listdir(dirx):
    if file!=".DS_Store" and os.path.isfile(os.path.join(dirx, file)):
        shr.append(Image.open(os.path.join(dirx, file)))

In [16]:
rw,cl = np.asarray(shr[0]).shape[:2]
decrypt_share(shr,rw,cl,c,"Deshare_DES")

# DeSharing TDES

In [17]:
dirx = "/Users/himanshuchuri/Desktop/Cryptography/Project/Churi-Doshi-Pawar-Final/Save_TDES_dec/"

In [18]:
shr,c = [],n
for file in os.listdir(dirx):
    if file!=".DS_Store" and os.path.isfile(os.path.join(dirx, file)):
        shr.append(Image.open(os.path.join(dirx, file)))

In [19]:
rw,cl = np.asarray(shr[0]).shape[:2]
decrypt_share(shr,rw,cl,c,"Deshare_TDES")